In [1]:
%%capture
!pip install llama-index==0.10.37 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-cohere==0.2.0

In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

In [5]:
API_KEY_CO= os.getenv("API_KEY_COHERE")
API_KEY_OPEN_AI = os.getenv("API_KEY_OPEN_AI")
API_KEY_QDRANT = os.getenv("API_KEY_QDRANT")

In [9]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files = [r"C:\Users\anteb\Desktop\AGENTIC RETRIEVAL-AUGMENTED GENERATION A SURVEY ON AGENTIC RAG.pdf"],
    filename_as_id=True).load_data()